In [2]:
# 1. Instalação das dependências
!pip install -q transformers accelerate bitsandbytes

import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ==========================================
# CONFIGURAÇÃO DE MODELO (MISTRAL V3)
# ==========================================
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

print(f"Carregando {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# ==========================================
# LEITURA DOS 3 ARQUIVOS
# ==========================================
def ler_arquivo(nome):
    if os.path.exists(nome):
        with open(nome, "r", encoding="utf-8") as f:
            return f.read()
    return f"Arquivo {nome} não encontrado."

print("Lendo arquivos...")
conteudo_releases = ler_arquivo("releases.txt")


# ==========================================
# CONSTRUÇÃO DO PROMPT (FORMATO MISTRAL)
# ==========================================
prompt_texto = f"""Analise os dados do projeto Screenpipe para entender o fluxo de trabalho.

--- DADOS DE RELEASES ---
{conteudo_releases[:5000]}


TAREFA:
1. Identifique e descreva as estratégias de release.
2. Identifique e descreva o modelo de fluxo de trabalho (ex: GitFlow, Trunk-based).
Dê uma resposta técnica e estruturada."""

# O Mistral v3 usa [INST] e [/INST]
prompt_final = f"[INST] {prompt_texto} [/INST]"

# ==========================================
# EXECUÇÃO DA ANÁLISE
# ==========================================
print("\n--- INICIANDO ANÁLISE INTEGRADA (MISTRAL) ---\n")

saida = pipe(prompt_final, max_new_tokens=1200, do_sample=False, return_full_text=False)

print("\n=== RESULTADO DA ANÁLISE ===\n")
print(saida[0]['generated_text'])

Carregando mistralai/Mistral-7B-Instruct-v0.3...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Lendo arquivos...

--- INICIANDO ANÁLISE INTEGRADA (MISTRAL) ---


=== RESULTADO DA ANÁLISE ===

 1. **Release Strategy**: Screenpipe appears to follow a Semantic Versioning (SemVer) strategy, as indicated by the format of the release tags (major.minor.patch). This strategy allows for clear communication about the nature of changes in each release, with major releases indicating backward-incompatible changes, minor releases indicating new features or improvements, and patch releases indicating bug fixes.

2. **Workflow Model**: Based on the commit messages and the structure of the repository, it seems that Screenpipe is using a GitHub Flow or GitFlow-like workflow. This workflow involves developing features on feature branches, creating pull requests for code review, merging approved pull requests into a `develop` branch, and then periodically merging the `develop` branch into the `master` branch to create a new release.

In more detail:

- Developers create feature branches from the `